# LSTM-arithmetic

## Dataset
- [Arithmetic dataset](https://drive.google.com/file/d/1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-/view?usp=sharing)

In [1]:
# ! pip install seaborn
# ! pip install opencc
# ! pip install -U scikit-learn

import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
import matplotlib.pyplot as plt
import seaborn as sns
import opencc
import os
from sklearn.model_selection import train_test_split

data_path = './data'

In [2]:
df_train = pd.read_csv(os.path.join(data_path, 'arithmetic_train.csv'))
df_eval = pd.read_csv(os.path.join(data_path, 'arithmetic_eval.csv'))
df_train.head()
# df_eval.head()

,src,tgt
0,14*(43+20)=,882
1,(6+1)*5=,35
2,13+32+29=,74
3,31*(3-11)=,-248
4,24*49+1=,1177


In [3]:
# transform the input data to string
df_train['tgt'] = df_train['tgt'].apply(str)
df_train['full_sequence'] = df_train['src'] + df_train['tgt']
df_train['input_text'] = df_train['full_sequence'].apply(lambda s: s[:-1])
df_train['target_text'] = df_train['full_sequence'].apply(lambda s: s[1:])


# testing set
df_eval['tgt'] = df_eval['tgt'].apply(lambda x: str(x))
df_eval['full_answer'] = df_eval['src'] + df_eval['tgt']

In [4]:
# df_train.head()
df_eval.head()

,src,tgt,full_answer
0,48+43+34=,125,48+43+34=125
1,30-(48+13)=,-31,30-(48+13)=-31
2,(21*31)+10=,661,(21*31)+10=661
3,2-27-10=,-35,2-27-10=-35
4,(15*20)+24=,324,(15*20)+24=324


# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [23]:
char_to_id = {}
id_to_char = {}

# write your code here
# Build a dictionary and give every token in the train dataset an id
# The dictionary should contain <eos> and <pad>
# char_to_id is to conver charactors to ids, while id_to_char is the opposite
char_to_id['<pad>'] = 0
char_to_id['<eos>'] = 1
longest_src_sequence_length = 0
id = 2
for src_string in df_train['full_sequence']:
    for c in src_string:
        if (len(src_string) > longest_src_sequence_length):
            longest_src_sequence_length = len(src_string)
        if (not char_to_id.get(c)):
            char_to_id[c] = id
            id += 1
            
print(char_to_id)
id_to_char = { v: k for k, v in char_to_id.items()}
print(id_to_char)

vocab_size = len(char_to_id)
print('Vocab size {}'.format(vocab_size))

{'<pad>': 0, '<eos>': 1, '1': 2, '4': 3, '*': 4, '(': 5, '3': 6, '+': 7, '2': 8, '0': 9, ')': 10, '=': 11, '8': 12, '6': 13, '5': 14, '9': 15, '7': 16, '-': 17}
{0: '<pad>', 1: '<eos>', 2: '1', 3: '4', 4: '*', 5: '(', 6: '3', 7: '+', 8: '2', 9: '0', 10: ')', 11: '=', 12: '8', 13: '6', 14: '5', 15: '9', 16: '7', 17: '-'}
Vocab size 18


# Data Preprocessing
 - The data is processed into the format required for the model's input and output. (End with \<eos\> token)


In [24]:
# Write your code here
# add paddiing and eos char first

def string_to_list(curr_string, max_length):
    str_list = list(curr_string)
    original_length = len(curr_string)
    str_list += ["<pad>"] * (max_length - original_length) + ["<eos>"]
    
    return str_list

def list_to_id(token_list):
    return [char_to_id[token] for token in token_list]
    
df_train['tgt_list'] = df_train['target_text'].apply(lambda s: string_to_list(s, longest_src_sequence_length))
df_train['tgt_list_id'] = df_train['tgt_list'].apply(lambda s : list_to_id(s))
df_train['src_list'] = df_train['input_text'].apply(lambda s: string_to_list(s, longest_src_sequence_length))
df_train['src_list_id'] = df_train['src_list'].apply(lambda s : list_to_id(s))
df_train.head()


,src,tgt,full_sequence,input_text,target_text,tgt_list,tgt_list_id,src_list,src_list_id
0,14*(43+20)=,882,14*(43+20)=882,14*(43+20)=88,4*(43+20)=882,"[4, *, (, 4, 3, +, 2, 0, ), =, 8, 8, 2, <pad>,...","[3, 4, 5, 3, 6, 7, 8, 9, 10, 11, 12, 12, 8, 0,...","[1, 4, *, (, 4, 3, +, 2, 0, ), =, 8, 8, <pad>,...","[2, 3, 4, 5, 3, 6, 7, 8, 9, 10, 11, 12, 12, 0,..."
1,(6+1)*5=,35,(6+1)*5=35,(6+1)*5=3,6+1)*5=35,"[6, +, 1, ), *, 5, =, 3, 5, <pad>, <pad>, <pad...","[13, 7, 2, 10, 4, 14, 11, 6, 14, 0, 0, 0, 0, 0...","[(, 6, +, 1, ), *, 5, =, 3, <pad>, <pad>, <pad...","[5, 13, 7, 2, 10, 4, 14, 11, 6, 0, 0, 0, 0, 0,..."
2,13+32+29=,74,13+32+29=74,13+32+29=7,3+32+29=74,"[3, +, 3, 2, +, 2, 9, =, 7, 4, <pad>, <pad>, <...","[6, 7, 6, 8, 7, 8, 15, 11, 16, 3, 0, 0, 0, 0, ...","[1, 3, +, 3, 2, +, 2, 9, =, 7, <pad>, <pad>, <...","[2, 6, 7, 6, 8, 7, 8, 15, 11, 16, 0, 0, 0, 0, ..."
3,31*(3-11)=,-248,31*(3-11)=-248,31*(3-11)=-24,1*(3-11)=-248,"[1, *, (, 3, -, 1, 1, ), =, -, 2, 4, 8, <pad>,...","[2, 4, 5, 6, 17, 2, 2, 10, 11, 17, 8, 3, 12, 0...","[3, 1, *, (, 3, -, 1, 1, ), =, -, 2, 4, <pad>,...","[6, 2, 4, 5, 6, 17, 2, 2, 10, 11, 17, 8, 3, 0,..."
4,24*49+1=,1177,24*49+1=1177,24*49+1=117,4*49+1=1177,"[4, *, 4, 9, +, 1, =, 1, 1, 7, 7, <pad>, <pad>...","[3, 4, 3, 15, 7, 2, 11, 2, 2, 16, 16, 0, 0, 0,...","[2, 4, *, 4, 9, +, 1, =, 1, 1, 7, <pad>, <pad>...","[8, 3, 4, 3, 15, 7, 2, 11, 2, 2, 16, 0, 0, 0, ..."


In [25]:
print(longest_src_sequence_length)

16


In [27]:
print(df_train['src_list_id'][3])

[6, 2, 4, 5, 6, 17, 2, 2, 10, 11, 17, 8, 3, 0, 0, 0, 1]


# Hyper Parameters

|Hyperparameter|Meaning|Value|
|-|-|-|
|`batch_size`|Number of data samples in a single batch|64|
|`epochs`|Total number of epochs to train|10|
|`embed_dim`|Dimension of the word embeddings|256|
|`hidden_dim`|Dimension of the hidden state in each timestep of the LSTM|256|
|`lr`|Learning Rate|0.001|
|`grad_clip`|To prevent gradient explosion in RNNs, restrict the gradient range|1|

In [ ]:
BATCH_SIZE = 128
EPOCHS = 2
EMBED_DIM = 8
HIDDEN_DIM = 256
LR = 0.001
GRAD_CLIP = 1

# nohup python -u training_script.py > b128_e2_em8_h256_lr0.001.log &

# Data Batching
- Use `torch.utils.data.Dataset` to create a data generation tool called  `dataset`.
- The, use `torch.utils.data.DataLoader` to randomly sample from the `dataset` and group the samples into batches.

- Example: 1+2-3=0
    - Model input: 1 + 2 - 3 = 0
    - Model output: / / / / / 0 &lt;eos&gt;  (the '/' can be replaced with &lt;pad&gt;)
    - The key for the model's output is that the model does not need to predict the next character of the previous part. What matters is that once the model sees '=', it should start generating the answer, which is '0'. After generating the answer, it should also generate&lt;eos&gt;

In [29]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
    
    def __len__(self):
        # return the amount of data
        return len(self.dataframe)
    
    def __getitem__(self, index):
        # Extract the input data x and the ground truth y from the data
        x = self.dataframe['src_list_id'].iloc[index]
        y = self.dataframe['tgt_list_id'].iloc[index]
        return x, y

# collate function, used to build dataloader
def collate_fn(batch):
    batch_x = [torch.tensor(data[0]) for data in batch]
    batch_y = [torch.tensor(data[1]) for data in batch]
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])
    
    # Pad the input sequence
    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens

In [30]:
ds_train = Dataset(df_train)
ds_test = Dataset(df_eval)

In [31]:
# Build dataloader of train set and eval set, collate_fn is the collate function
dl_train = torch.utils.data.DataLoader(
    ds_train, 
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
    shuffle=True
)
dl_test = torch.utils.data.DataLoader(
    ds_test, 
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
    shuffle=False
)

# Model Design

## Execution Flow
1. Convert all characters in the sentence into embeddings.
2. Pass the embeddings through an LSTM sequentially.
3. The output of the LSTM is passed into another LSTM, and additional layers can be added.
4. The output from all time steps of the final LSTM is passed through a Fully Connected layer.
5. The character corresponding to the maximum value across all output dimensions is selected as the next character.

## Loss Function
Since this is a classification task, Cross Entropy is used as the loss function.

## Gradient Update
Adam algorithm is used for gradient updates.

In [32]:
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()
        
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])
        
        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.linear = torch.nn.Sequential(torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=hidden_dim),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=vocab_size))
        
    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)
    
    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        batch_x = self.embedding(batch_x)
        
        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)
        
        batch_x, _ = self.rnn_layer1(batch_x)
        batch_x, _ = self.rnn_layer2(batch_x)
        
        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x,
                                                            batch_first=True)
        
        batch_x = self.linear(batch_x)
        
        return batch_x
    
    def generator(self, start_char, max_len=200):
        
        char_list = [char_to_id[c] for c in start_char]
        
        self.eval()
        
        next_char = None
        with torch.no_grad():
            while len(char_list) < max_len: 
                # Write your code here 
                # Pack the char_list to tensor
                # Input the tensor to the embedding layer, LSTM layers, linear respectively
                input_tensor = torch.tensor(char_list, dtype=torch.long).unsqueeze(0)
                input_tensor = input_tensor.to(next(self.parameters()).device)
                
                embedded = self.embedding(input_tensor)
                rnn_out, _ = self.rnn_layer1(embedded)
                rnn_out, _ = self.rnn_layer2(rnn_out)
                
                last_token_logit = self.linear(rnn_out[:, -1, :])
                next_char_id = torch.argmax(last_token_logit, dim=1).item()
                
                if next_char_id == char_to_id['<eos>']:
                    break
                
                char_list.append(next_char_id)
            
        return [id_to_char[ch_id] for ch_id in char_list]

In [33]:
torch.manual_seed(2)

print(vocab_size)
device = 'cuda'

model = CharRNN(vocab_size,
                EMBED_DIM,
                HIDDEN_DIM)

18


In [34]:
padding_idx = char_to_id['<pad>']
criterion = torch.nn.CrossEntropyLoss(ignore_index=padding_idx)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

# Training
1. The outer `for` loop controls the `epoch`
    1. The inner `for` loop uses `data_loader` to retrieve batches.
        1. Pass the batch to the `model` for training.
        2. Compare the predicted results `batch_pred_y` with the true labels `batch_y` using Cross Entropy to calculate the loss `loss`
        3. Use `loss.backward` to automatically compute the gradients.
        4. Use `torch.nn.utils.clip_grad_value_` to limit the gradient values between `-grad_clip` &lt; and &lt; `grad_clip`.
        5. Use `optimizer.step()` to update the model (backpropagation).
2.  After every `1000` batches, output the current loss to monitor whether it is converging.

In [ ]:
from tqdm import tqdm
from copy import deepcopy
model = model.to(device)
model.train()
i = 0
for epoch in range(1, EPOCHS+1):
    # The process bar
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        # Write your code here
        # Clear the gradient
        optimizer.zero_grad()
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_pred_y = model(batch_x, batch_x_lens)
        
        # Write your code here
        # Input the prediction and ground truths to loss function
        # Back propagation
        loss = criterion(batch_pred_y.view(-1, vocab_size), batch_y.view(-1))
        loss.backward()
        
        torch.nn.utils.clip_grad_value_(model.parameters(), GRAD_CLIP) # gradient clipping

        optimizer.step()
        # Write your code here
        # Optimize parameters in the model

        i+=1
        if i%50==0:
            bar.set_postfix(loss = loss.item())
    
    # Evaluate your model
    model.eval()
    matched = 0
    total = 0
    bar_eval = tqdm(df_eval.iterrows(), desc=f"Validation epoch {epoch}")
    for _, row in bar_eval:
        batch_x = row['src']
        batch_y = row['tgt']
        
        prediction = model.generator(batch_x)# An example of using generator: model.generator('1+1=')
        
        if prediction == batch_y:
            matched += 1
        total += 1
        # Write your code here. Input the batch_x to the model and generate the predictions
        # Write your code here.
        # Check whether the prediction match the ground truths
        # Compute exact match (EM) on the eval dataset
        # EM = correct/total

        
    em_accuracy = matched / total if total > 0 else 0
    print(f"Validation Epoch {epoch} | Exact Match Accuracy: {em_accuracy:.4f}")

Train epoch 1:   0%|          | 0/2369250 [00:00<?, ?it/s]

Train epoch 1:   0%|          | 8081/2369250 [00:27<2:13:11, 295.45it/s, loss=1.44]


KeyboardInterrupt: 